In [4]:
import socket
import struct
import time

### Connection
# Connect to a Betaflight drone via TCP
host = '192.168.1.252'  # IP address of the drone
port = 5761  # port typically used for TCP-based connections with Betaflight

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))


In [5]:
### MSP commands
# out messages usually start with 1 (1**)
MSP_STATUS = 101
MSP_RAW_IMU = 102
# in messages usually start with 2 (2**)
MSP_SET_MOTOR = 214

In [6]:
### Main functions
def construct_msp_message(command: int, data: list = []) -> bytearray:
    """Creates expected formatting.

    Command frame format:
    Header: `$M<`
    Body: length of `data` + `command` + `data` values
    Tail: checksum

    Args:
        command (int): MSP command
        data (list, optional): list of params (length is varying). Defaults to [].

    Returns:
        bytearray: command frame
    """
    message = bytearray()
    # header
    message.append(ord('$'))
    message.append(ord('M'))
    message.append(ord('<'))
    # body
    message.append(len(data))
    message.append(command)
    checksum = len(data) ^ command
    for byte in data:
        message.append(byte)
        checksum ^= byte
    # tail
    message.append(checksum)
    return message

def send_msp_message(command: int, data: list = []):
    """Send prepared command via serial

    Args:
        command (int): MSP command
        data (list, optional): list of params. Defaults to [].
    """
    message = construct_msp_message(command, data)
    sock.sendall(message)

In [7]:
def read_msp_response() -> tuple:
    """Read data

    Returns:
        tuple: command code and its values
    """
    header = sock.recv(3)  # read header
    if header != b'$M>':
        return None
    # read body
    data_length = sock.recv(1)[0]
    code = sock.recv(1)[0]
    data = sock.recv(data_length)
    # read tail
    checksum = sock.recv(1)[0]
    # check if the command is of the expected length
    calculated_checksum = data_length ^ code
    for byte in data:
        calculated_checksum ^= byte
    if calculated_checksum != checksum:
        return None

    return (code, data)

### Get data functions
def get_raw_imu() -> tuple:
    """Read raw IMU and decode it

    Returns:
        tuple | None: tuple - processed data, None - received data doesn't correspond to sent command
    """
    send_msp_message(MSP_RAW_IMU)  # send request
    time.sleep(0.005)  # wait for reply
    response = read_msp_response() # get reply

    if response and response[0] == MSP_RAW_IMU:
        imu = struct.unpack(f'<{len(response[1]) // 2}H', response[1])  # decode
        return imu
    return None

In [ ]:

#MSP_SENS = 108  # yaw works better
MSP_SENS = 143

### Get data functions
def get_sensor() -> tuple:
    """Read raw IMU and decode it

    Returns:
        tuple | None: tuple - processed data, None - received data doesn't correspond to sent command
    """
    send_msp_message(MSP_SENS )  # send request
    #time.sleep(0.005)  # wait for reply
    response = read_msp_response() # get reply

    if response and response[0] == MSP_SENS:
        alt = struct.unpack(f'<{len(response[1]) // 2}H', response[1])  # decode
        return alt
    return None

In [65]:
while True:
    print(get_sensor())
    time.sleep(0.1)

BrokenPipeError: [Errno 32] Broken pipe

In [8]:
### Set motors values functions
def set_motor_values(motor1: int, motor2: int, motor3: int, motor4: int):
    """Set throttle per motor

    Args:
        motor1 (int): motor 1 throttle
        motor2 (int): motor 2 throttle
        motor3 (int): motor 3 throttle
        motor4 (int): motor 4 throttle
    """
    motor_values = [motor1, motor2, motor3, motor4, 0, 0, 0, 0]
    send_msp_message(MSP_SET_MOTOR, struct.pack('<8H', *motor_values))

In [15]:
set_motor_values(0,0,0,0)

accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, mag_x, mag_y, mag_z

In [ ]:
def get_raw_imu() -> tuple:
    """Read raw IMU data and decode it as signed 16-bit integers.

    Returns:
        tuple | None: tuple - processed data, None - received data doesn't correspond to sent command
    """
    send_msp_message(MSP_RAW_IMU)  # send request
    time.sleep(0.009)  # wait for reply
    response = read_msp_response()  # get reply

    if response and response[0] == MSP_RAW_IMU:
        # Decode as signed 16-bit integers
        imu = struct.unpack('<9h', response[1])  # AccX, AccY, AccZ, GyroX, GyroY, GyroZ
        #imu = struct.unpack(f'<{len(response[1]) // 2}H', response[1])  # decode
        return imu
    return None


In [ ]:
while True:
    print(get_raw_imu())

In [ ]:
import time
import numpy as np

# Constants
g = 9.81  # Gravitational acceleration in m/s²
dt = 0.01  # Time step in seconds (adjust based on your loop timing)
small_accel_threshold = 150.0  # Threshold for acceleration in millig

# Initial conditions
position_x = 0.0  # Initial position in X
position_y = 0.0  # Initial position in Y
position_z = 0.0  # Initial position in Z
velocity_x = 0.0  # Initial velocity in X
velocity_y = 0.0  # Initial velocity in Y
velocity_z = 0.0  # Initial velocity in Z

def estimate_position():
    global position_x, position_y, position_z
    global velocity_x, velocity_y, velocity_z

    while True:
        raw_imu_data = get_raw_imu()
        if raw_imu_data:
            acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z, _, _, _ = raw_imu_data

            # Check and update X-axis
            if abs(acc_x) > small_accel_threshold:
                if acc_x > 0:
                    print("pos_x")
                else:
                    print("neg_x")


            # Check and update Y-axis
            if abs(acc_y) > small_accel_threshold:
                if acc_y > 0:
                    print("pos_y")
                else:
                    print("neg_y")

            # Print position for demonstration
            #print(f"Estimated Position: X={position_x:.2f} m, Y={position_y:.2f} m, Z={position_z:.2f} m")

        # Uncomment to control loop timing
        time.sleep(dt)

# Start the position estimation
estimate_position()


In [ ]:
import time
import numpy as np

# Constants
g = 9.81  # Gravitational acceleration in m/s²
dt = 0.001  # Time step in seconds (adjust based on your loop timing)
x_accel_threshold = 400.0  # Threshold for acceleration in millig
y_accel_threshold = 400.0  # Threshold for acceleration in millig
small_value_adjustment = 0.1  # Small value to adjust positions

# Initial conditions
position_x = 0.0  # Initial position in X
position_y = 0.0  # Initial position in Y
position_z = 0.0  # Initial position in Z

def estimate_position():
    global position_x, position_y, position_z

    while True:
        raw_imu_data = get_raw_imu()
        if raw_imu_data:
            acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z, _, _, _ = raw_imu_data

            # Check and update X-axis
            if abs(acc_x) > x_accel_threshold:
                print(acc_x)
                if acc_x > 0:
                    position_x += small_value_adjustment  # Increment position
                    #print("Incrementing position_x")
                else:
                    position_x -= small_value_adjustment  # Decrement position
                    #print("Decrementing position_x")

            # Print positions for demonstration
            #print(f"Estimated Position: X={position_x:.2f} m, Y={position_y:.2f} m, Z={position_z:.2f} m")


        # Uncomment to control loop timing
        time.sleep(dt)

# Start the position estimation
estimate_position()
